# Задание 1 - Классификация

* Необходимо предсказать клас y на фичах a, b, c.

* Задание 1. Построить модель Pymc с множественной логистической регрессией для решения задачи. Предсказать класс на test_clf.

* Задание 2. Построить модель Pymc с порождающим множественным логистическим классификатором для решения задачи. Предсказать класс на test_clf.

* Каждое задание оценивается в 10 баллов

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

import scipy.stats as stats
import pymc as pm
import arviz as az

from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Задание 2

* Задание 1. Востановить параметры линейной регрессии (ее вид и веса) с использованием Pymc

* Задание 2. Найти для тестовых X значения y предсказанного для вашей отобраной регрессии.

In [2]:
df = pd.read_csv('train_reg.csv')
df.head(3)

,x,y
0,0.415709,2.267527
1,2.078132,5.761260
2,6.152402,17.237352


In [3]:
X_train, X_test, y_train, y_test = train_test_split( df['x'], df['y'], 
                                                    test_size=0.3, 
                                                    random_state=42)

In [4]:
with pm.Model() as model:
    α = pm.Normal('α', mu=0, sigma=10) 
    β = pm.Normal('β', mu=0, sigma=1) 
    ε = pm.HalfNormal('ε', 10)
    μ = pm.Deterministic('μ', α + β * X_train)
    y_pred = pm.Normal('y_pred', mu=μ, sigma=ε, observed=y_train)
    trace = pm.sample(2000, tune=2000, chains=4, cores=4)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [α, β, ε]


Sampling 4 chains for 2_000 tune and 2_000 draw iterations (8_000 + 8_000 draws total) took 18 seconds.
/Users/imachome/anaconda3/lib/python3.11/site-packages/arviz/utils.py:185: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  numba_fn = numba.jit(**self.kwargs)(self.function)


In [5]:
summary_df = az.summary(trace)
summary_df

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
α,-0.876,0.400,-1.641,-0.136,0.006,0.005,3874.0,3818.0,1.0
β,3.113,0.061,2.991,3.221,0.001,0.001,3888.0,4004.0,1.0
ε,1.815,0.156,1.530,2.113,0.002,0.002,4637.0,4362.0,1.0
μ[0],-4.059,0.454,-4.946,-3.241,0.007,0.005,3786.0,3678.0,1.0
μ[1],21.290,0.241,20.859,21.763,0.003,0.002,9001.0,6084.0,1.0
...,...,...,...,...,...,...,...,...,...
μ[65],35.531,0.441,34.737,36.391,0.006,0.004,5468.0,5386.0,1.0
μ[66],9.713,0.253,9.238,10.190,0.004,0.002,5131.0,5162.0,1.0
μ[67],13.423,0.225,13.006,13.856,0.003,0.002,6632.0,5386.0,1.0
μ[68],1.987,0.354,1.308,2.639,0.006,0.004,4013.0,3804.0,1.0


In [6]:
α = summary_df.head(1).iloc[:,0].item()
β = summary_df.iloc[1:2].iloc[:,0].item()


y_pred = α + β * X_test

y_pred

83    17.658164
53     1.211561
70    11.590562
45    -0.878609
44    22.778012
39    21.094785
22    27.772828
80    -0.713282
10    25.048572
0      0.418103
18    28.942238
30    26.492606
73     6.166409
33    22.435839
90    22.633210
4     24.466861
76    19.599517
77    -0.062798
12    32.683565
31     5.408243
55     7.456541
88    15.546911
26    33.126497
42    17.893503
69     9.648623
15    17.558159
40    25.791841
96    -1.417311
9     22.694841
72    10.178706
Name: x, dtype: float64

In [7]:
print(mean_squared_error(y_test, y_pred))
print(mean_absolute_error(y_test, y_pred))

1.9690106385341066
1.2013957451404897


2

In [8]:
X_test = pd.read_csv('test_reg.csv')
X_test.head(3)

,x
0,5.913738
1,2.657985
2,-0.666239


In [9]:
y_predict = α + β * X_test

y_predict

,x
0,17.533467
1,7.398307
2,-2.950002
3,29.915580
4,21.613007
...,...
95,27.743674
96,32.077503
97,4.866558
98,0.798502


In [10]:
from sklearn.metrics import mean_squared_error
y_true = pd.read_csv('test_Y_true_reg.csv')
mean_squared_error(y_true, y_predict)

2.34755391377849